# Worksheet 08

Name:  Hao Qi  
UID:  U96305250  

### Topics

- Soft Clustering
- (optional) Clustering Aggregation

### Probability Review

Read through [the following material](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8).

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.426919766209902, 4.5473247310663325, 4.9276809346223285, 5.917724818537847, 5.504288309037737, 4.420727344214572, 5.364293556890076, 4.896385598409866, 5.131422926973593, 6.355050465655652]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.513094345554559, 8.550914438792216, 7.014739087852444, 8.769019964132038, 9.375229374244553, 7.546633933499634, 8.05723466048525, 7.908885540921297, 7.269672175743532, 8.668387866911555]


b) Flip a fair coin 10 times. If the coin lands on H, pick the last data point of `s1` and remove it from `s1`; if T pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip a coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.355050465655652, 5.131422926973593, 4.896385598409866, 8.668387866911555, 5.364293556890076, 7.269672175743532, 7.908885540921297, 4.420727344214572, 8.05723466048525, 7.546633933499634]


c) `data` has a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Number of mixture components (Gaussian distributions): 2
- Info of Gaussian distribution I: mean=5, variance=1
- Info of Gaussian distribution II: mean=8, variance=1
- Expected proportion of mixture components: P(s1)=0.5, P(s2)=0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the actual values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))] # ??? a bit arbitrary
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2))]
mean = [sum(s1) / len(s1), sum(s2) / len(s2)]
var = [sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1), sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.668387866911555, 7.269672175743532, 7.908885540921297, 8.05723466048525, 7.546633933499634]
[6.355050465655652, 5.131422926973593, 4.896385598409866, 5.364293556890076, 4.420727344214572]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.890162835512254,  mean_2 = 5.233575978428751
var_1 = 0.227383699854461,  var_2 = 0.4119295222524741


/Users/paul/anaconda3/envs/msai/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


- We predetermined the number of clusters to be 2, consistent with the number of mixture components.  
- `mean_1` = 7.89; we speculate this is the initialization information for the distribution with the larger mean among the two Gaussian distributions. Therefore, this value is close to the actual value of 8.  
- `mean_2` = 5.23; this value is relatively far from the actual value of 5.  
- `var_1` = 0.23; the estimated variance for the first cluster is much smaller than 1. `var_2` = 0.41; the estimated variance for the second cluster is also lower than the actual value of 1. 
- The proportions of the two clusters (`P(S_1)`, `P(S_2)`) meet our expectations.  

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth? 

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        # pdf_i.append(norm.pdf(p, mean[j], var[j])) 
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5)) 
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0]*prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1]*prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.355050465655652
probability of observing that point if it came from cluster 0 =  0.004699476825339877
probability of observing that point if it came from cluster 1 =  0.13505283259199924
point =  5.131422926973593
probability of observing that point if it came from cluster 0 =  4.513400838277542e-08
probability of observing that point if it came from cluster 1 =  0.6137587582256289
point =  4.896385598409866
probability of observing that point if it came from cluster 0 =  2.308421461336597e-09
probability of observing that point if it came from cluster 1 =  0.5414563305092014
point =  8.668387866911555
probability of observing that point if it came from cluster 0 =  0.2208822956650832
probability of observing that point if it came from cluster 1 =  3.7519640456333065e-07
point =  5.364293556890076
probability of observing that point if it came from cluster 0 =  6.756676460351857e-07
probability of observing that point if it came from cluster 1 =  0.608823220497502
point =  7

In [6]:
truth = []
for p in data:
    if p in s1:
        truth.append(1)
    elif p in s2:
        truth.append(2)
    else:
        print("ERROR")
print(truth)

probs = zip(data, prob_s0_x, prob_s1_x)
prediction = []
for p in probs:
    if p[1] > p[2]:
        prediction.append(1)    # the distribution with a lager mean value
    else:
        prediction.append(2)
print(prediction)

[2, 2, 2, 1, 2, 1, 1, 2, 1, 1]
[2, 2, 2, 1, 2, 1, 1, 2, 1, 1]


The calculated probabilities have effectively suggested each data point's probable cluster allocation, generally matching the actual cluster from the original data creation process. I will assign each point to the cluster with a higher probability. Our allocation is now consistent with the truth from the results above. To enhance the credibility of this experiment, we may add more data points and repeat the process with different random seeds for sampling. 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j` and `P(S_j)`. How different are these values from the original ones you got from K means? Briefly comment.

In [7]:
prob_c = [sum(prob_s0_x)/len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.881359332849096,  mean_2 = 5.2313453239892205
var_1 = 0.24125625994090955,  var_2 = 0.4162863717952426


- The proportions of the two clusters (`P(S_1)`, `P(S_2)`) stay unchanged.  
- `mean_1` is slightly lower but the change is not obvious. The fluctuation of `mean_2` is even smaller. 
- Both `var_1` and `var_2` are a little larger. I guess the numbers randomly selected this time are relatively concentrated and centered around these two mean values. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        # pdf_i.append(norm.pdf(p, mean[j], var[j]))
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0]*prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1]*prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.355050465655652
probability of observing that point if it came from cluster 0 =  3.3665615441419434e-09
probability of observing that point if it came from cluster 1 =  0.025076647386591915
point =  5.131422926973593
probability of observing that point if it came from cluster 0 =  1.013620578554398e-28
probability of observing that point if it came from cluster 1 =  0.9311223879397578
point =  4.896385598409866
probability of observing that point if it came from cluster 0 =  9.486785367585139e-34
probability of observing that point if it came from cluster 1 =  0.6933102990281881
point =  8.668387866911555
probability of observing that point if it came from cluster 0 =  0.008082538278978958
probability of observing that point if it came from cluster 1 =  1.5095883653626116e-15
point =  5.364293556890076
probability of observing that point if it came from cluster 0 =  3.817359302523639e-24
probability of observing that point if it came from cluster 1 =  0.9106884829079049
poin

In [9]:
truth = []
for p in data:
    if p in s1:
        truth.append(1)
    elif p in s2:
        truth.append(2)
    else:
        print("ERROR")
print(truth)

probs = zip(data, prob_s0_x, prob_s1_x)
prediction = []
for p in probs:
    if p[1] > p[2]:
        prediction.append(1)    # the distribution with a lager mean value
    else:
        prediction.append(2)
print(prediction)

[2, 2, 2, 1, 2, 1, 1, 2, 1, 1]
[2, 2, 2, 1, 2, 1, 1, 2, 1, 1]


Our allocation is still consistent with the truth. But each data point now holds a probability of 0.99 or 1 for cluster membership, showing strong certainty. This illustrates the adaptive aspect of this clustering algorithm, which fine-tunes predictions with the given data, despite the tiny change here. 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [15]:
probs = zip(data, prob_s0_x, prob_s1_x)
assignment = zip(data, [0 if p[1] > p[2] else 1 for p in probs])
print([[p, i] for p, i in assignment])

[[6.355050465655652, 1], [5.131422926973593, 1], [4.896385598409866, 1], [8.668387866911555, 0], [5.364293556890076, 1], [7.269672175743532, 0], [7.908885540921297, 0], [4.420727344214572, 1], [8.05723466048525, 0], [7.546633933499634, 0]]
